In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from test_helper import *

from scipy.spatial.distance import cdist
from matplotlib import gridspec

import os 
import textwrap as tw
from PIL import Image

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
tf.compat.v1.disable_eager_execution()

In [3]:
left_input_im = tf.compat.v1.placeholder(tf.float32, [None, 256, 128, 3], 'left_input_im')
right_input_im = tf.compat.v1.placeholder(tf.float32, [None, 256, 128, 3], 'right_input_im')
left_label = tf.compat.v1.placeholder(tf.float32, [None, ], 'left_label')
right_label = tf.compat.v1.placeholder(tf.float32, [None, ], 'right_label')

# print(np.shape(left_input_im), np.shape(right_input_im))
logits, model_left, model_right = inference(left_input_im, right_input_im)
global_step = tf.compat.v1.Variable(0, trainable=False)
global_init = tf.compat.v1.variables_initializer(tf.compat.v1.global_variables())

(None, 256, 128, 3)
(None, 256, 128, 256)
(None, 128, 64, 256)
(None, 128, 64, 128)
(None, 64, 32, 128)
(None, 64, 32, 64)
(None, 32, 16, 64)
(None, 32, 16, 32)
(None, 16, 8, 32)
(None, 4096)
(None, 4096)
(None, 256, 128, 3)
(None, 256, 128, 256)
(None, 128, 64, 256)
(None, 128, 64, 128)
(None, 64, 32, 128)
(None, 64, 32, 64)
(None, 32, 16, 64)
(None, 32, 16, 32)
(None, 16, 8, 32)
(None, 4096)
(None, 4096)


In [4]:
saver = tf.compat.v1.train.Saver()
with tf.compat.v1.Session() as sess:
    sess.run(global_init)
    ckpt = tf.train.get_checkpoint_state("model")
    saver.restore(sess, "model_siamese/model.ckpt")

    for im1 in range(1,8):
        for im2 in range(1,8):


            img = Image.open(str(im1) + '.jpg')
            img = np.array(img)[np.newaxis,:,:,:]
            # img2 = img
            img2 = Image.open(str(im2) + '.jpg')
            img2 = np.array(img2)[np.newaxis,:,:,:]


            my_logits, model_lf, model_rg = sess.run([logits, model_left, model_right], \
                                                     feed_dict={left_input_im:img, right_input_im:img2})
            
            lft = np.array(model_lf[0])
            rgt = np.array(model_rg[0])
            l = lft - rgt

            distance = np.sqrt(np.sum((l) ** 2))
            similarity = my_logits * np.square(distance)  # keep the similar label (1) close to each other
            dissimilarity = (1 - np.array(my_logits[0])) * np.square(np.max((0.5 - distance),
                                                           0))  # give penalty to dissimilar label if the distance is bigger than margin
            similarity_loss = np.mean(dissimilarity + similarity) / 2

            dist = cdist(model_lf, model_rg, 'cosine')
            euc = np.linalg.norm(model_lf - model_rg)

            fig = plt.figure()
            plt.title(('Similarity: %f, Dissimilarity: %f\nEuclidean Dist: %f, Logits: %f' % (similarity, dissimilarity, euc, my_logits)), loc='center')
            if my_logits > 0.0:
                textstr = 'Similar ' + str(my_logits)
                props = dict(boxstyle='round', facecolor='green', alpha=0.5)
                fig_txt = tw.fill(tw.dedent(textstr), width=80)
                plt.figtext(0.51, 0.05, fig_txt, horizontalalignment='center',
                            fontsize=12, multialignment='center',
                            bbox=dict(boxstyle="round", facecolor='green',
                                      ec="0.5", pad=0.5, alpha=1), fontweight='bold')
            else:
                textstr = 'Dissimilar ' + str(my_logits)
                props = dict(boxstyle='round', facecolor='red', alpha=0.5)
                fig_txt = tw.fill(tw.dedent(textstr), width=80)
                plt.figtext(0.51, 0.05, fig_txt, horizontalalignment='center',
                            fontsize=12, multialignment='center',
                            bbox=dict(boxstyle="round", facecolor='red',
                                      ec="0.5", pad=0.5, alpha=1), fontweight='bold')



            plt.axis('off')
            ax1 = fig.add_subplot(1, 2, 1)
            l_im = np.array(img)[0]
            ax1.imshow(l_im)
            ax1.axis('off')
            ax2 = fig.add_subplot(1, 2, 2)
            r_im = np.array(img2)[0]
            ax2.imshow(r_im)
            ax2.axis('off')


            plt.show()





2024-04-15 18:19:01.191549: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


ValueError: The passed save_path is not a valid checkpoint: model_siamese/model.ckpt